In [19]:
from fastapi import FastAPI
import pandas as pd
import json
from fastapi.middleware.cors import CORSMiddleware
import os

app = FastAPI(title="GenAI Transaction Intelligence API")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],   # allow all origins (dev mode)
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

INSIGHT_FILE = "../data/transaction_insights.json"
TXN_FILE = "../data/transactions_patterns.csv"


df = pd.read_csv(TXN_FILE)

with open(INSIGHT_FILE) as f:
    all_insights = json.load(f)

# -----------------------------


In [20]:
### Prompts
SYSTEM_PROMPT = """
You are a banking transaction explanation assistant.

Rules:
- Explain ONLY using the provided facts.
- Do NOT add new information.
- Do NOT speculate about user intent.
- Do NOT provide financial advice.
- Keep explanations factual and concise.
- Maximum 3 sentences.
"""

USER_PROMPT_TEMPLATE = """
Explain the following transaction insight in clear, human-friendly language.

Insight Type: {insight_type}
Severity: {severity}
Facts:
{facts}

Explanation:
"""

In [21]:
import requests

OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "llama3.1:8b"

def explain_insight(insight: dict) -> str:

    facts_text = "\n".join(
        [f"- {k}: {v}" for k, v in insight["facts"].items()]
    )

    prompt = USER_PROMPT_TEMPLATE.format(
        insight_type=insight["insight_type"],
        severity=insight["severity"],
        facts=facts_text
    )

    payload = {
        "model": MODEL_NAME,
        "prompt": f"{SYSTEM_PROMPT}\n{prompt}",
        "stream": False,
        "options": {
            "temperature": 0.2,
            "num_predict": 120
        }
    }

    response = requests.post(OLLAMA_URL, json=payload)
    response.raise_for_status()

    return response.json()["response"].strip()


In [22]:
def get_transactions(limit: int = 50):
    return df.head(limit).to_dict(orient="records")

def explain_transaction(txn_id: str):
    txn = df[df["transaction_id"] == txn_id]
    if txn.empty:
        return {"error": "Transaction not found"}

    txn_insights = [
        i for i in all_insights
        if i["transaction_id"] == txn_id
    ]

    explanations = []

    for insight in txn_insights:
        text = explain_insight(insight)
        explanations.append({
            "insight_type": insight["insight_type"],
            "explanation": text
        })

    return {
        "transaction": txn.iloc[0].to_dict(),
        "explanations": explanations
    }



In [23]:

data = explain_transaction("TXN000889")

In [24]:
data


{'transaction': {'transaction_id': 'TXN000889',
  'user_id': 'U001',
  'datetime': '2026-02-10 01:11:47.874388',
  'amount': 21.83,
  'merchant': 'RAPIDKL',
  'category': 'Transport',
  'channel': 'E-WALLET',
  'payment_type': 'DEBIT',
  'location': 'Kuala Lumpur',
  'hour': 1,
  'day_of_week': 1,
  'is_weekend': False,
  'time_bucket': 'LATE_NIGHT',
  'category_avg_amount': 28.41,
  'category_std_amount': 12.1,
  'category_txn_count': 120,
  'merchant_avg_amount': 27.09,
  'merchant_txn_count': 38,
  'amount_zscore': -0.54,
  'flags': 'LATE_NIGHT_WEEKDAY',
  'reasons': 'Transaction occurred late at night on a weekday'},
 'explanations': [{'insight_type': 'TIME_BEHAVIOR_SHIFT',
   'explanation': "This transaction occurred during late night hours (between midnight and 2am) on a weekday, specifically at 1am. It's not related to a weekend or holiday period. This timing may be unusual for the user's typical behavior."}]}